# Предмет "Применение NLP в Здравоохранении"

## Гладышев В.В.

## Подготовка базы ответов для бота - курсовика по NLP

In [1]:
core_numbers = 4
w2v_width = 300
ft_width = 300

In [2]:
%pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32897 sha256=2bba8968d0407436087466aace6e22e327c275d837afbd1f1e0db638d5d2da51
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [3]:
%pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552448 sha256=a7e48eab0629c1bf818c51d1e95a01aff602f8711b41813115f6b99c88e9b583
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [4]:
%pip install compress_fasttext

  Preparing metadata (setup.py) ... done
  Created wheel for compress_fasttext: filename=compress_fasttext-0.1.5-py3-none-any.whl size=16098 sha256=e348b0cc5eeb18f2aae655ee274c5ee0b0f26ac5599cc2ce840b56f2937827d1
  Stored in directory: /root/.cache/pip/wheels/8c/32/33/fff40f9f82948020c3cee0e392223360212af37a60f1b207e6
Successfully built compress_fasttext


In [7]:
%pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 54.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=5ab6cd4ee8d272b8187e7f2365af4518a48785993ba03796a4fd4b08483659ed
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [8]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText, KeyedVectors
import compress_fasttext
from gensim.models.fasttext import FastTextKeyedVectors
import pickle
import numpy as np
from tqdm import tqdm    # tqdm.notebook
from pathlib import Path
from linecache import getline
import re

## Загружаем исходный корпус вопросов и ответов

In [ ]:
disk = Storage.ya_disk(application_id='8b22c6d569724e6b92xxxxxxxxxxx', application_secret='b2b380d359a54802bexxxxxxxxxxxx')

In [ ]:
disk.get('Jupyter/NLP/Otvety.txt', 'Otvety.txt')

/usr/local/lib/python3.7/dist-packages/ml_kernel/kernel.py:442: UserWarning: The following variables cannot be serialized: disk
  warnings.warn(message)


In [ ]:
disk.get('Jupyter/NLP/ft_freqprune_400K_100K_pq_300.bin', 'ft_freqprune_400K_100K_pq_300.bin')

/usr/local/lib/python3.7/dist-packages/ml_kernel/kernel.py:442: UserWarning: The following variables cannot be serialized: disk
  warnings.warn(message)


## Word2vec эмбеддинг

In [11]:
question = None
written = False

with open("prepared_answers2.txt", "w") as fout:
    with open("Otvety.txt", "r", errors='ignore') as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                line = re.sub(r'<br>','\n', line)
                line = re.sub(r'<[^<]*>','', line)
                if(len(line)>4076):
                    tmp_line = line[:4076]
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

2it [00:00, 2166.48it/s]


In [12]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [13]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
flag_w2v = False
flag_ft = False

w2v_file = Path("word2vec_wv.bin")
if w2v_file.is_file():
#     model_w2v_wv = KeyedVectors.load(w2v_file, mmap='r')
    model_w2v_wv = KeyedVectors.load_word2vec_format(w2v_file, binary=True)
#     modelW2V = Word2Vec.load(w2v_file)
    flag_w2v = True

# ft_file = Path("fasttext_wv.bin")
ft_file = Path("ft_freqprune_400K_100K_pq_300.bin")
if ft_file.is_file():
#     modelFT = FastText.load("fasttext.model")
    model_ft_wv = FastTextKeyedVectors.load("ft_freqprune_400K_100K_pq_300.bin")
    flag_ft = True

In [ ]:
model_ft_wv["Парацетамол"]

array([-0.15978006,  0.12972822,  0.21127554,  0.29666308,  0.25573889,
        0.1686621 , -0.05744876, -0.52107803,  0.18609457, -0.22283846,
       -0.03836431, -0.04170422, -0.10437255, -0.29092596,  0.10090196,
        0.33342996, -0.67721425, -0.15884957,  0.02521641, -0.33641117,
       -0.26925578,  0.23259699,  0.13870712, -0.08089827, -0.03441673,
        0.24143436, -0.29945117,  0.12255987,  0.09575692, -0.08634345,
       -0.21093656, -0.34836296, -0.17476538,  0.45353659, -0.10380922,
       -0.09972098,  0.09274811,  0.06449159, -0.06760033, -0.10698365,
       -0.11544589, -0.39024976, -0.16776919, -0.02356622,  0.19175797,
       -0.19430752, -0.00745214,  0.37378223,  0.06579063, -0.17072464,
        0.14309801,  0.14525867, -0.82332072, -0.25561934, -0.00373324,
        0.08323687, -0.18829364,  0.18259489, -0.24611599,  0.15786773,
       -0.27496507,  0.34677877,  0.1449238 ,  0.06140086, -0.08831784,
       -0.04187009, -0.56405255, -0.04610748, -0.28585907,  0.05

In [14]:
def get_w2vembed_and_ftembed(file_name):
    questions = []
    with open(file_name, "r", encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f):
            spls = line.split("\t")
            questions.append(preprocess_txt(spls[0]))

    modelW2V = Word2Vec(sentences=questions, size=w2v_width, window=5, min_count=1, workers=core_numbers)
#     modelFT = FastText(sentences=questions, size=w2v_width, min_count=1, window=5, workers=core_numbers)

#     modelW2V.save("word2vec.model")
    modelW2V.wv.save_word2vec_format('word2vec_wv.bin', binary=True)
#     modelW2V.wv.save("word2vec_wv.kv")
    model_w2v_wv = modelW2V.wv
    del modelW2V

#     modelFT.save("fasttext.model")
#     modelFT.wv.save_word2vec_format('fasttext_wv.bin', binary=True)
#     model_ft_wv = modelFT.wv
#     del modelFT

    return model_w2v_wv # , model_ft_wv

In [ ]:
if not flag_w2v: # or not flag_ft:
    model_w2v_wv = get_w2vembed_and_ftembed(Path("prepared_answers.txt"))

In [ ]:
# modelW2V.wv.save_word2vec_format('word2vec_wv.bin', binary=True)

In [ ]:
w2v_index = annoy.AnnoyIndex(w2v_width ,'angular')
ft_index = annoy.AnnoyIndex(ft_width ,'angular')

In [ ]:
# flag_index_map = False
flag_w2v_index = False
flag_ft_index = False

In [ ]:
# index_map = {}

# index_file = Path("index_map.pickle")
# if index_file.is_file():
#     with open('index_map.pickle', 'rb') as f:
#         index_map = pickle.load(f)
#     flag_index_map = True

w2v_index_file = Path("w2v_index.ann")
if w2v_index_file.is_file():
    w2v_index.load("w2v_index.ann")
    flag_w2v_index = True

ft_index_file = Path("ft_index.ann")
if ft_index_file.is_file():
    ft_index.load("ft_index.ann")
    flag_ft_index = True

In [ ]:
type(model_ft_wv)

gensim.models.keyedvectors.FastTextKeyedVectors

In [ ]:
if not flag_w2v_index or not flag_ft_index:  # or not flag_index_map:
    counter = 0

    with open("prepared_answers.txt", "r", encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f):
            n_w2v = 0
            n_ft = 0
            spls = line.split("\t")
#             index_map[counter] = spls[1]
            question = preprocess_txt(spls[0])

            vector_w2v = np.zeros(w2v_width)
            vector_ft = np.zeros(ft_width)
            for word in question:
                if word in model_w2v_wv:
                    vector_w2v += model_w2v_wv[word]
                    n_w2v += 1
                if word in model_ft_wv:
                    vector_ft += model_ft_wv[word]
                    n_ft += 1
            if n_w2v > 0:
                vector_w2v = vector_w2v / n_w2v
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            w2v_index.add_item(counter, vector_w2v)
            ft_index.add_item(counter, vector_ft)

            counter += 1
#             if counter > 1800000:
#                 break
#             if not counter % 10000:
#                 print(counter)

    w2v_index.build(20)
    ft_index.build(20)
#     with open('index_map.pickle', 'wb') as fl:
#         pickle.dump(index_map, fl)
    w2v_index.save('w2v_index.ann')
    ft_index.save('ft_index.ann')

1163342it [1:46:54, 181.36it/s]


In [ ]:
# with open('index_map.pickle', 'wb') as fl:
#     pickle.dump(index_map, fl)

# w2v_index.save('w2v_index.ann')

In [ ]:
def get_response(question, index, model, width):
    question = preprocess_txt(question)
    vector = np.zeros(width)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    line_numbers = index.get_nns_by_vector(vector, 3)
    print(line_numbers)
#     f = open("prepared_answers.txt", "r", encoding='utf-8', errors='ignore')
    spls = [getline("prepared_answers.txt", i+1).split("\t") for i in line_numbers]
    for spl in spls:
        print(f"Вопрос: {spl[0]}\nОтвет: {spl[1]}")

In [ ]:
TEXT = "как понизить температуру?"

In [ ]:
get_response(TEXT, w2v_index, model_w2v_wv, w2v_width)

In [ ]:
get_response(TEXT, ft_index, model_ft_wv, ft_width)

In [ ]:
TEXT = "Когда принимать Працетамол?"

In [ ]:
get_response(TEXT, w2v_index, model_w2v_wv, w2v_width)

In [ ]:
get_response(TEXT, ft_index, model_ft_wv, ft_width)

## Сохраняем модель на Яндекс диск

In [ ]:
from cloud_ml.storage.api import Storage

# To retrieve application id and secret:
# 1. Go to link: https://oauth.yandex.ru/client/new
# 2. Choose 'Web services'
# 3. Paste into 'Callback URI': https://oauth.yandex.ru/verification_code
# 4. Set up permissions on yandex disk
disk = Storage.ya_disk(application_id='8b22c6d569724e6b92646b7d6d2a3953', application_secret='b2b380d359a54802be5c27e2360e2c08')

# uploading contents of the local file into the remote one
# disk.put('word2vec_wv.bin', 'Jupyter/NLP/full_300/word2vec_wv.bin')
# disk.put('fasttext_wv.bin', 'Jupyter/NLP/full_300/fasttext_wv.bin')
# disk.put('word2vec.model', 'Jupyter/NLP/m500k/word2vec.model')
# disk.put('word2vec.model.trainables.syn1neg.npy', 'Jupyter/NLP/m500k/word2vec.model.trainables.syn1neg.npy')
# disk.put('word2vec.model.wv.vectors.npy', 'Jupyter/NLP/m500k/word2vec.model.wv.vectors.npy')
# disk.put('w2v_index.ann', 'Jupyter/NLP/full_300/w2v_index.ann')
disk.put('ft_index.ann', 'Jupyter/NLP/full_300/ft_index.ann')
# disk.put('index_map.pickle', 'Jupyter/NLP/m1800k/index_map.pickle')
disk.put('prepared_answers2.txt', 'Jupyter/NLP/prepared_answers2.txt')

/usr/local/lib/python3.7/dist-packages/ml_kernel/kernel.py:442: UserWarning: The following variables cannot be serialized: disk
  warnings.warn(message)
